# Filtre de spam basé sur "Naive Bayes Classifier" 📧😡

😁 N-K`
<br><br>

Sommaire :
 - <a href="#part1">Présentation des données</a>
 - <a href="#part2">Chargement des données</a> 
 - <a href="#part3">L'analyse exploratoire des données</a> 
 - <a href="#part4">Préparation du classifieur Naïve Bayes</a>
 - <a href="#part5">Entrainement du Naive Bayes Classifier</a> 
 - <a href="#part5">Tester et valider le Spam Filter</a>
<br><br><br>

## Présentation des données
Ce notebook contient un jeux de données correspondants à :
- Colonne 1 : emails
- Colonne 2 : spam ( 1 si "oui" 0 si "non")

## Importation des libraries

In [14]:
import numpy as np
import pandas as pd
import nltk # Natural Language Toolkit (NLTK) est une plate-forme leader pour la création de programmes Python pour travailler avec des données de langage humain
from nltk.corpus import stopwords
import string

## Chargement des données

In [15]:
data = pd.read_csv('emails.csv')
data.head()

,text,spam
0,Subject: naturally irresistible your corporate...,1
1,Subject: the stock trading gunslinger fanny i...,1
2,Subject: unbelievable new homes made easy im ...,1
3,Subject: 4 color printing special request add...,1
4,"Subject: do not have money , get software cds ...",1


## L'analyse exploratoire des données

In [16]:
print("Describe :")
print(data.describe())
print('-'*20)
print("\n Infos :")
print(data.info())
print('-'*20)
print("\n Valeurs manquantes :")
print(data.isnull().sum())
print('-'*20)
print("\n Redondance :")
print(data[data.duplicated() == True].shape)
print('-'*20)

Describe :
              spam
count  5728.000000
mean      0.238827
std       0.426404
min       0.000000
25%       0.000000
50%       0.000000
75%       0.000000
max       1.000000
--------------------

 Infos :
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 5728 entries, 0 to 5727
Data columns (total 2 columns):
 #   Column  Non-Null Count  Dtype 
---  ------  --------------  ----- 
 0   text    5728 non-null   object
 1   spam    5728 non-null   int64 
dtypes: int64(1), object(1)
memory usage: 89.6+ KB
None
--------------------

 Valeurs manquantes :
text    0
spam    0
dtype: int64
--------------------

 Redondance :
(33, 2)
--------------------


##### Donc nous n'avons pas de valeurs manquantes mais par contre nous avons des lignes répétées (33 lignes)

In [17]:
# suppression des lignes repetées
data.drop_duplicates(inplace = True)

In [18]:
data.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 5695 entries, 0 to 5727
Data columns (total 2 columns):
 #   Column  Non-Null Count  Dtype 
---  ------  --------------  ----- 
 0   text    5695 non-null   object
 1   spam    5695 non-null   int64 
dtypes: int64(1), object(1)
memory usage: 133.5+ KB


## Préparation du classifieur Naïve Bayes
<br><br>
Avant il faut faire les traitements necessaires sur le texte :
 - <a href="#part1">Suppression des ponctuations</a>
 - <a href="#part2">Suppression des stopwords</a> 
 - <a href="#part3">Retourner un texte prêt pour le traitement</a> 
<br>

### Suppression des ponctuations

In [21]:
nltk.download('stopwords')

[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\user\AppData\Roaming\nltk_data...
[nltk_data]   Unzipping corpora\stopwords.zip.


True

In [24]:
#1.Les ponctuations sont [!"#$%&'()*+,-./:;<=>?@[\]^_`{|}~]
#2.Les stopwords dans le traitement du langage naturel, sont des mots inutiles (données).

def traitement_text(text):
    
     #1 Supprimer la ponctuationa
        nopunc = [char for char in text if char not in string.punctuation]
        nopunc = ''.join(nopunc)
    
     #2 Supprimer les mots vides
        clean_words= [word for word in nopunc.split() if word.lower() not in stopwords.words('english')]
    
     #3 Renvoie une liste de mots clairs
        return clean_words

In [27]:
#Afficher la tokenisation (une liste de tokens)
data['text'].head().apply(traitement_text)

0    [Subject, naturally, irresistible, corporate, ...
1    [Subject, stock, trading, gunslinger, fanny, m...
2    [Subject, unbelievable, new, homes, made, easy...
3    [Subject, 4, color, printing, special, request...
4    [Subject, money, get, software, cds, software,...
Name: text, dtype: object

#### Il faut, pour chaque email, compter le nombre d'occurrences de mots. Cela permettra de calculer la probabilité conditionnelle.

#### Il s'agit de la méthode fit_transform de la classe countVectorizer qui effectue ce calcul sur Python.

In [26]:
from sklearn.feature_extraction.text import CountVectorizer

In [28]:
courriel = CountVectorizer(analyzer=traitement_text).fit_transform(data['text'])

## Entrainement du Naive Bayes Classifier

In [31]:
from sklearn.model_selection import train_test_split

In [32]:
#Distribuer les données à 80% pour entainer le modèle ML et 20% pour le test
X_train, X_test, y_train, y_test = train_test_split(courriel, data['spam'], test_size = 0.2, random_state = 0)

#### Scikit_learn nous propose la classe MultinomialNB (NB pour Naïve Bayes) par la suite:
- On donne à MultinomialNB() la version codifiée des mails qu’on a calculé avec CountVectorizer
- Pour chaque mail, on spécifie s’il s’agit de SPAM ou non

In [35]:
from sklearn.naive_bayes import MultinomialNB

In [36]:
classifier = MultinomialNB().fit(X_train,y_train)

In [37]:
#Afficher les valeurs prédites
print(classifier.predict(X_train))
#Afficher les valeurs actuelles
print(y_train.values)

[0 0 0 ... 0 0 0]
[0 0 0 ... 0 0 0]


#### ALors on peut dire que les valeurs prédites correspondent aux valeurs réelles. 

## Tester et valider le Spam Filter

In [40]:
from sklearn.metrics import classification_report, confusion_matrix, accuracy_score

In [44]:
val_pred = classifier.predict(X_test)
print(classification_report(y_test,val_pred))

              precision    recall  f1-score   support

           0       1.00      0.99      0.99       870
           1       0.97      1.00      0.98       269

    accuracy                           0.99      1139
   macro avg       0.98      0.99      0.99      1139
weighted avg       0.99      0.99      0.99      1139



In [45]:
print("Matrice de confusion : \n", confusion_matrix(y_test,val_pred))

Matrice de confusion : 
 [[862   8]
 [  1 268]]


In [46]:
print("Précision du modèle : \n", accuracy_score(y_test,val_pred))

Précision du modèle : 
 0.9920983318700615


### ALors on vient d’implémenter un Spam Filter avec une precision de 99.2%  🥳🎉🎊